# Setting up

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Lab/Grain-U-Net'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))
sys.path.append('/content/drive/MyDrive/{}/src'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/drive
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-7d3f4fee4ec3>", line 10, in <module>
    get_ipython().magic('cd /content/drive/My\\ Drive/$FOLDERNAME')
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-84>", line 2, in cd
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd()
OSError: [Errno 107] Transport endpoint is not 

OSError: ignored

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport -tensorflow
!pip install bayesian-optimization  
!pip install -q plantcv
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content


# Impoting Data

In [ ]:
from pathlib import Path
import numpy as np

train_dir = Path('data/train_atarashii_256')

input_names = list(train_dir.glob('**/image/*.png'))
label_names = list(train_dir.glob('**/label/*.png'))

print(f"Found {len(input_names)} samples and {len(label_names)} tracings")

Found 675 samples and 675 tracings


In [ ]:
from src import GrainSequence
import random

def createSequences(batch_size): 
    validation_samples = len(input_names) // 10

    random.Random(1337).shuffle(input_names)
    random.Random(1337).shuffle(label_names)

    train_input = input_names[:-validation_samples]
    train_label = label_names[:-validation_samples]
    train_gen = GrainSequence(batch_size, (256,256), train_input, train_label)

    valid_input = input_names[-validation_samples:]
    valid_label = label_names[-validation_samples:]
    valid_gen = GrainSequence(batch_size, (256,256), valid_input, valid_label)

    print(f"Training set size: {len(train_input)}, {len(train_gen)} batches")
    print(f"Validation set size: {len(valid_input)}, {len(valid_gen)} batches")

Training set size: 608, 608 batches
Validation set size: 67, 67 batches


# Creating the model

In [ ]:
import numpy as np 
from keras.optimizers import *
from src import get_unet_hyper

def fit_with(dropout_rate, lr, epochs, batch_size, verbose=1):
    epochs = int(epochs)
    batch_size = int(batch_size)
    model = get_unet_hyper(dropout_rate=dropout_rate)

    createSequences(batch_size)

    opt = keras.optimizers.Adam(learning_rate = lr)
    model.compile(optimizer = opt, loss='binary_crossentropy', metrics = ['accuracy'])

    model.fit(x=train_gen, steps_per_epoch=len(train_gen), epochs=epochs, batch_size=batch_size, verbose=verbose)

    #add in test set
    score=model.evaluate(valid_gen , steps=len(valid_gen), verbose=0)

    print('Test loss: ', score[0])
    print('Test accuracy: ', score[1])

    return score[1]

In [ ]:
from tensorflow import keras
from bayes_opt import BayesianOptimization
import functools

#dropout_rate, lr, epochs, batch_size
pbounds = {'dropout_rate': (0.1, 0.8), 
           'lr': (1e-4, 1e-2), 
           'epochs': (25, 60), 
           'batch_size': (1, 32)
           }


In [ ]:
optimizer = BayesianOptimization(
    f=fit_with, 
    pbounds=pbounds, 
    verbose=2, 
    random_state=1
)

optimizer.maximize(init_points=10, n_iter=10)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   | batch_... | dropou... |  epochs   |    lr     |
-------------------------------------------------------------------------
Epoch 1/25
  3/608 [..............................] - ETA: 1:15:01 - loss: 2585853696.0000 - accuracy: 0.8327

KeyboardInterrupt: ignored